# Import Modules

In [349]:
import pandas as pd
import re, time, requests, random
from datetime import datetime
# 爬蟲
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# import grequests # 看起要在.py才能用

# 非同步HTTP請求
from aiohttp import ClientSession, TCPConnector
# Python標準庫中提供的用於支援異步編程的模組, 可使用async和await關鍵字來定義異步協程
import asyncio

# 處理嵌套事件迴圈（nested event loop）的庫, 在Python中，通常只能有一個事件迴圈運行，
# 但某些情況下，比如在Jupyter Notebook中執行異步代碼時，可能會遇到嵌套事件迴圈的問題。
# nest_asyncio的作用就是解決這個問題，允許在已有事件迴圈的情況下再創建一個新的事件迴圈。
import nest_asyncio
# 它會修改當前執行環境，允許在已有事件迴圈的情況下再次建立一個新的事件迴圈，
# 通常用於處理一些特定的情況，例如在Jupyter Notebook中執行異步代碼。
nest_asyncio.apply()

# reload之後再執行 網路爬蟲 Web crawler

In [438]:
# 測試連結
link = "https://www.104.com.tw/jobs/search/?ro=1&indcat=1002000000&zone=4,5&edu=4,5&isnew=3&"
link = "https://www.104.com.tw/jobs/search/?ro=1&indcat=1002000000&zone=4,5&edu=4,5"
link ="https://www.104.com.tw/jobs/search/?ro=1&zone=4,5&edu=4,5,6&jobexp=3,5,10,99&"
excludeJobKeyword = "excludeJobKeyword=軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶"

excludeJobKeyword2 = "excludeJobKeyword=軟體,會計,品管,採購,資訊"

link = "https://www.104.com.tw/jobs/search/?ro=1&indcat=1001000000,1002000000,1012000000,1014000000&zone=4,5&"
excludeJobKeyword3 = "excludeJobKeyword=軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶"

link = link + excludeJobKeyword3


# link = link+"excludeJobKeyword="+"軟體,行政,會計,百貨,資訊,品管,財務,守衛,技術員,法務,品檢"
print(link)

https://www.104.com.tw/jobs/search/?ro=1&indcat=1001000000,1002000000,1012000000,1014000000&zone=4,5&excludeJobKeyword=軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶


In [542]:
import crawler104, config.search_params, async_example, threaded_async_job, jobs104
import importlib
importlib.reload(crawler104)
importlib.reload(async_example)
importlib.reload(threaded_async_job)
importlib.reload(jobs104)
importlib.reload(config.search_params)
from config.search_params import get_filter_params

# custom filter search
role = {'ro':'全職'}
keyword = {'keyword':"後端工程師 python"}
area = {'area':['新北市', '台北市', '桃園市', '台中市']}
isnew = {'isnew':'三日內'}
jobexp = {'jobexp':['1年以下', '1-3年']}

# 預設
mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
order = {'order':'日期排序'}
asc = {'asc':'遞減'}

filter_params = get_filter_params(role, keyword, area, isnew, jobexp, mode, order, asc)
filter_params

{'ro': 1,
 'keyword': '後端工程師 python',
 'area': '6001002000,6001001000,6001005000,6001008000',
 'isnew': 3,
 'jobexp': '1,3',
 'mode': 'l',
 'order': 16,
 'asc': 0}

In [482]:
dict = {'ro': '全職'}
key_to_check = next(iter(dict), None)
key_to_check

'ro'

In [547]:
from crawler104 import Crawler104
import async_job
from config.search_params import get_filter_params
import numpy as np
import pandas as pd
import time


# custom filter search
role = {'ro':'全職'}
keyword = {'keyword':"後端工程師 python"}
area = {'area':['新北市', '台北市', '桃園市', '台中市']}
isnew = {'isnew':'三日內'}
jobexp = {'jobexp':['1年以下', '1-3年']}

# 預設
mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
order = {'order':'日期排序'}
asc = {'asc':'遞減'}

filter_params = get_filter_params(role, keyword, area, isnew, jobexp, mode, order, asc)

# for simon
# filter_params = {'ro': 1,
#  'zone': '4,5',
#  'mode': 'l',
#  'indcat':'1001000000,1002000000,1012000000,1014000000',
#  'order': 16,
#  'asc': 0,
#  'excludeJobKeyword': '軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶' 
#                 }

# 碼表 Start
start_time = time.time()
# 建立df物件
df = pd.DataFrame()
crawler104 = Crawler104(filter_params, keywords_pattern)
# 開始爬蟲
raw_jobs_104 = crawler104.search_job()
filter_jobs_104 = crawler104.filter_job(raw_jobs_104)
# result_df = js104.main(filter_jobs, df)
# 碼表 End
print(f"花費 {np.round((time.time() - start_time),2)} 秒")

url: https://www.104.com.tw/jobs/search/?ro=1&keyword=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB+python&area=6001002000%2C6001001000%2C6001005000%2C6001008000&isnew=3&jobexp=1%2C3&mode=l&order=16&asc=0


Loading: 100%|██████████| 68/68 [01:48<00:00,  1.59s/page]


載入68頁 | 載入2018筆資料 | 過濾後剩2018筆資料 | 花費 124.0 秒


In [546]:
import crawler104, config.search_params, async_example, threaded_async_job, jobs104
import importlib
importlib.reload(crawler104)
importlib.reload(async_example)
importlib.reload(threaded_async_job)
importlib.reload(jobs104)
importlib.reload(config.search_params)

<module 'config.search_params' from 'C:\\Users\\Rekam\\Documents\\Jobscan\\config\\search_params.py'>

# 異步&多線程 網頁抓取 Web scraper

In [356]:
filter_jobs = filter_jobs_104[200:280]

In [ ]:
# from crawler104 import Crawler104
# jobs_link = crawler104.get_jobs_link(filter_jobs)
# 網頁抓取 
results = threaded_async_job.scraper(filter_jobs_104)

jobs:2018
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
Batch Start
博晶醫電股份有限公司:Node.js全端工程師:2024/01/18
揚帆科技有限公司:Golang後端工程師:2024/01/18
水星工程科技有限公司:Vue前端網頁設計師:2024/01/18
柏昇企業股份有限公司:軟體工程師(Web 後端):2024/01/18
必勝客Pizza Hut/肯德基KFC_富利餐飲股份有限公司:集團總部 - DTS System Engineer , App System Developer:2024/01/17
嘉興資訊顧問有限公司:系統維運工程師:2024/01/18
慧友電子股份有限公司:軟韌體自動化測試開發工程師:2024/01/17
昊坤創業投資顧問股份有限公司:UI/UX設計師:2024/01/18
拓連科技股份有限公司:後端軟體工程師:2024/01/18
Inventec_英業達股份有限公司:[AI] 機器人工程師 / Robotics Engineer (士林):2024/01/17
誠雲科技股份有限公司:資安服務工程師-技術支援(北部辦公室):2024/01/18
弈樂科技股份有限公司:後端系統開發工程師:2024/01/1

In [298]:
column_name = '工作內容'
# pd.set_option('display.max_colwidth', None)

selected_column  = results[0][0][1][column_name]
df = results[0][0][1]
print(selected_column)


0    - 徵求在學學生或應屆畢業生\r\n- 區塊鏈相關 WEB APP 開發與維護。\r\n- ITRI、Apple、Tesla 技術高手親自指導，幫助你學習國外最新 SaaS 開發的觀念與思維。\r\n- 學習量化交易知識與免費使用公司自動交易服務。\r\n- 表現優異者，有機會轉 Paid Intern 或正職！\r\n※ 請注意：此為學習型無薪實習，為期三個月，可配合學校簽訂契約、實習證明和推薦信（表現需具備一定水準者），無法接受者，請勿投遞，謝謝！\r\n※ 需通過遠端線下 coding 測驗（CSS + React 各一題）
Name: 工作內容, dtype: object


In [91]:
data_list1 = [{'@context': 'http://schema.org', '@type': 'BreadcrumbList', 'itemListElement': [{'@type': 'ListItem', 'position': 1, 'name': '104人力銀行', 'item': 'https://www.104.com.tw/'}, {'@type': 'ListItem', 'position': 2, 'name': '雲育鏈股份有限公司', 'item': 'https://www.104.com.tw/company/1a2x6bld2j'}, {'@type': 'ListItem', 'position': 3, 'name': 'Python 開發工程師(台北)', 'item': 'https://www.104.com.tw/job/87e35'}]}, {'@context': 'http://schema.org', '@type': 'WebPage', 'name': 'Python 開發工程師(台北)', 'url': '//www.104.com.tw/job/87e35', 'relatedLink': 'https://www.104.com.tw/company/1a2x6bld2j', 'publisher': {'@type': 'Organization', 'name': '雲育鏈股份有限公司'}, 'significantLink': ['https://www.104.com.tw/company/1a2x6bld2j#info06']}, {'@context': 'http://schema.org', '@type': 'JobPosting', 'title': 'Python 開發工程師(台北)', 'url': '//www.104.com.tw/job/87e35?jobsource=google_job', 'datePosted': '2024/01/16', 'validThrough': '2025-01-17', 'description': '&lt;h3&gt;【工作內容】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;雲育鏈成立至今 4 年，致力於為台灣企業、金控提供有效雲端數位轉型方案，培訓了破千位的雲端、軟體人才，為台灣創造更多新型態企業與就業機會。&lt;br&gt;雲育鏈提供 AWS、GCP 雲端課程，並協助企業上雲。&lt;br&gt;合作的廠商包含金融產業、系統、遊戲、電信大廠等，提供內訓課程也提供一般標準化的課程。&lt;br&gt;&lt;br&gt;【工作內容】&lt;br&gt;設計和開發 Python 應用程式：Python 開發工程師的主要職責是設計和開發 Python 應用程式，包括 Web 應用程式、數據分析應用程式、機器學習應用程式等。&lt;br&gt;編寫高品質的程式碼：Python 開發工程師需要編寫高品質、易於維護的程式碼，並且了解程式碼編寫的最佳實踐和設計模式。&lt;br&gt;測試和調試程式碼：Python 開發工程師需要測試和調試程式碼，並且使用相應的測試框架和工具進行自動化測試。&lt;br&gt;整合第三方庫和 API：Python 開發工程師需要整合第三方庫和 API，以擴展 Python 的功能和特性，提高開發效率。&lt;br&gt;數據分析和機器學習：Python 是數據分析和機器學習領域的重要語言，Python 開發工程師需要了解這些領域的基礎知識和相關工具，並且能夠應用這些知識開發相應的應用程式。&lt;br&gt;敏捷開發方法論：Python 開發工程師需要了解敏捷開發方法論和相應的工具，如 Scrum、Kanban 等，以實現項目快速迭代和交付。&lt;br&gt;團隊協作能力：Python 開發工程師需要具備良好的團隊協作能力，能夠與其他開發人員、項目經理和客戶進行有效的溝通和協調，共同完成項目的開發和交付。&lt;br&gt;&lt;br&gt;- 職務類別：軟體工程師,其他資訊專業人員,軟體專案主管&lt;br&gt;- 工作待遇：待遇面議&lt;br&gt;- 工作性質：全職&lt;br&gt;- 上班地點：台北市中山區&lt;br&gt;- 管理責任：不需負擔管理責任&lt;br&gt;- 出差外派：需出差，一年累積時間約一個月以下&lt;br&gt;- 上班時段：日班/晚班&lt;br&gt;- 休假制度：週休二日&lt;br&gt;- 可上班日：不限&lt;br&gt;- 需求人數：1~2人&lt;br&gt;&lt;br&gt;&lt;h3&gt;【相關條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 工作經歷：2年以上&lt;br&gt;- 學歷要求：專科、大學、碩士&lt;br&gt;- 公司產業：其它軟體及網路相關業&lt;br&gt;- 科系要求：不拘&lt;br&gt;- 語文條件：不拘&lt;br&gt;- 擅長工具：Linux,Java,Python,AWS&lt;br&gt;- 工作技能：軟體工程系統開發,軟體程式設計,網路程式設計,資料庫系統管理維護,資料庫程式設計&lt;br&gt;&lt;br&gt;&lt;h3&gt;【其他條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 其他：1. Python 編程能力：Python 工程師需要熟悉 Python 語言的基礎語法、標準庫和常用的第三方庫，能夠使用 Python 編程開發各種類型的應用程式。&lt;br&gt;2. Web 開發：Python 是 Web 開發的重要語言之一，Python 工程師需要了解 Web 開發相關的知識，包括 Web 框架、HTTP 協議、前端開發等。&lt;br&gt;3. 數據分析和機器學習：Python 是數據分析和機器學習領域的重要語言之一，Python 工程師需要了解數據分析和機器學習的基礎知識和相關庫，如 NumPy、Pandas、Scikit-learn 等。&lt;br&gt;4. 敏捷開發方法論：Python 工程師需要了解敏捷開發方法論和相應的工具，如 Scrum、Kanban 等，以實現項目快速迭代和交付。&lt;br&gt;5. 資料庫：Python 工程師需要了解資料庫相關的知識，如 SQL 語言、關係型資料庫、NoSQL 資料庫等。&lt;br&gt;6. Linux/Unix 系統管理：Python 工程師需要了解 Linux/Unix 系統的基本操作和管理，能夠在 Linux/Unix 系統上部署和運行 Python 應用程式。&lt;br&gt;7. 基本的數學和統計學知識：Python 工程師需要了解基本的數學和統計學知識，以應用於數據分析和機器學習等領域。&lt;br&gt;&lt;br&gt;&lt;h3&gt;【公司福利】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 福利制度：1. 基本福利：勞健保、勞退&lt;br&gt;2. 經常性 Foodpanda 點心、飲料、宵夜&lt;br&gt;3. 公司零食&lt;br&gt;4. 歡樂輕鬆的辦公室氣氛&lt;br&gt;5. 經常性公司聚餐餐會&lt;br&gt;6. 季目標獎金 / 行銷 / 業務績效獎金&lt;br&gt;7. 不定期員工旅遊&lt;br&gt;8. 年終獎金&lt;br&gt;9. 有機會認識業界行銷、雲端、資訊、AWS原廠人脈&lt;br&gt;10. 有機會在業界曝光自己個人形象&lt;br&gt;11. 全體員工皆為「下午晚上」上班，可充分利用上午黃金時間&lt;br&gt;12. 公司尊重員工職涯發展意願&lt;br&gt;13. 員工與老闆皆對外誠實做自己&lt;br&gt;14. 特休&lt;br&gt;年終獎金,零食櫃,國內旅遊,部門聚餐,免費下午茶,下午茶&lt;br&gt;&lt;br&gt;&lt;br&gt;更多工作資訊請參考：https://www.104.com.tw/job/87e35', 'employmentType': 'FULL_TIME', 'hiringOrganization': {'@type': 'Organization', 'name': '雲育鏈股份有限公司', 'sameAs': 'https://www.104.com.tw/company/1a2x6bld2j', 'logo': 'https://static.104.com.tw/b_profile/cust_picture/7803/130000000157803/logo.png?v=20201011162918'}, 'identifier': {'@type': 'PropertyValue', 'name': '104人力銀行_一零四資訊科技股份有限公司', 'value': '87e35'}, 'jobLocation': {'@type': 'Place', 'address': {'@type': 'PostalAddress', 'streetAddress': '長安東路2段80號5F', 'addressLocality': '台北市中山區', 'postalCode': '104', 'addressRegion': '台灣', 'addressCountry': 'TW'}}, 'baseSalary': {'@type': 'MonetaryAmount', 'currency': 'TWD', 'value': {'@type': 'QuantitativeValue', 'unitText': 'MONTH', 'value': '40000元以上'}}, 'mainEntityOfPage': {'@type': 'WebPage', '@id': '//www.104.com.tw/job/87e35'}, 'educationRequirements': {'@type': 'EducationalOccupationalCredential', 'credentialCategory': 'professional certificate, bachelor degree, postgraduate degree'}, 'experienceRequirements': {'@type': 'OccupationalExperienceRequirements', 'monthsOfExperience': 24}, 'industry': '其它軟體及網路相關業', 'workHours': '日班/晚班', 'skills': ['軟體工程系統開發', '軟體程式設計', '網路程式設計', '資料庫系統管理維護', '資料庫程式設計']}]

In [94]:
data_list2 = [{'@context': 'http://schema.org', '@type': 'BreadcrumbList', 'itemListElement': [{'@type': 'ListItem', 'position': 1, 'name': '104人力銀行', 'item': 'https://www.104.com.tw/'}, {'@type': 'ListItem', 'position': 2, 'name': '奇策智能雲端股份有限公司', 'item': 'https://www.104.com.tw/company/1a2x6bliq7'}, {'@type': 'ListItem', 'position': 3, 'name': '※學習型※ 網站前、後端軟體研發工程師（Unpaid Internship）', 'item': 'https://www.104.com.tw/job/82ua3'}]}, {'@context': 'http://schema.org', '@type': 'WebPage', 'name': '※學習型※ 網站前、後端軟體研發工程師（Unpaid Internship）', 'url': '//www.104.com.tw/job/82ua3', 'relatedLink': 'https://www.104.com.tw/company/1a2x6bliq7', 'publisher': {'@type': 'Organization', 'name': '奇策智能雲端股份有限公司'}, 'significantLink': ['https://www.104.com.tw/company/1a2x6bliq7#info06']}, {'@context': 'http://schema.org', '@type': 'JobPosting', 'title': '※學習型※ 網站前、後端軟體研發工程師（Unpaid Internship）', 'url': '//www.104.com.tw/job/82ua3?jobsource=google_job', 'datePosted': '2024/01/16', 'validThrough': '2025-01-17', 'description': '&lt;h3&gt;【工作內容】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 徵求在學學生或應屆畢業生&lt;br&gt;- 區塊鏈相關 WEB APP 開發與維護。&lt;br&gt;- ITRI、Apple、Tesla 技術高手親自指導，幫助你學習國外最新 SaaS 開發的觀念與思維。&lt;br&gt;- 學習量化交易知識與免費使用公司自動交易服務。 &lt;br&gt;- 表現優異者，有機會轉 Paid Intern 或正職！ &lt;br&gt;&lt;br&gt;※ 請注意：此為學習型無薪實習，為期三個月，可配合學校簽訂契約、實習證明和推薦信（表現需具備一定水準者），無法接受者，請勿投遞，謝謝！&lt;br&gt;※ 需通過遠端線下 coding 測驗（CSS + React 各一題）&lt;br&gt;&lt;br&gt;&lt;br&gt;- 職務類別：Internet程式設計師,軟體工程師&lt;br&gt;- 工作待遇：論件計酬0元&lt;br&gt;- 工作性質：全職&lt;br&gt;- 上班地點：台北市中正區&lt;br&gt;- 管理責任：不需負擔管理責任&lt;br&gt;- 出差外派：無需出差外派&lt;br&gt;- 上班時段：日班/晚班/假日班，彈性配合&lt;br&gt;- 休假制度：週休二日&lt;br&gt;- 可上班日：不限&lt;br&gt;- 需求人數：不限&lt;br&gt;&lt;br&gt;&lt;h3&gt;【相關條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 工作經歷：不拘&lt;br&gt;- 學歷要求：高中以上&lt;br&gt;- 公司產業：電腦系統整合服務業&lt;br&gt;- 科系要求：資訊工程相關,其他數學及電算機科學相關&lt;br&gt;- 語文條件：英文--聽 /略懂、說 /略懂、讀 /精通、寫 /中等&lt;br&gt;- 擅長工具：Github,JavaScript,CSS,ReactJS&lt;br&gt;- 工作技能：軟體程式設計,網路程式設計&lt;br&gt;&lt;br&gt;&lt;h3&gt;【其他條件】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 其他：對以下領域有興趣者，歡迎應徵：&lt;br&gt;- Advanced Web Front-end Development&lt;br&gt;- Blockchain and Smart Contract&lt;br&gt;- Microservice Technology&lt;br&gt;- Machine Learning&lt;br&gt;- Cybersecurity&lt;br&gt;- 加密貨幣投資&lt;br&gt;&lt;br&gt;實習期間，你可以學習到：&lt;br&gt;- 技術分析&lt;br&gt;- 量化交易自動化&lt;br&gt;- 程式交易策略開發&lt;br&gt;- Agile / Scrum 開發實戰&lt;br&gt;- 前端原子設計框架&lt;br&gt;- React.js (Next.js)&lt;br&gt;- HTML / CSS (SCSS)&lt;br&gt;- TypeScript&lt;br&gt;- Node.js (ES6)&lt;br&gt;- Apollo GraphQL Client (react-graphql)&lt;br&gt;- ...&lt;br&gt;&lt;br&gt;&lt;br&gt;&lt;h3&gt;【公司福利】&lt;/h3&gt;&lt;br&gt;&lt;br&gt;- 福利制度：◎薪酬&lt;br&gt;•\t具市場競爭性的薪資&lt;br&gt;•\t員工免費股票&lt;br&gt;•\t員工低價認股權&lt;br&gt;•\t員工私募輪認購 Crypto-Arsenal Token 計畫&lt;br&gt;&lt;br&gt;◎工作、學習與生活並重&lt;br&gt;•\t遠距工作為主、定期面對面工作計畫與檢討為輔&lt;br&gt;•\t定期部門聚餐&lt;br&gt;•\t線上課程補助&lt;br&gt;&lt;br&gt;◎保險&lt;br&gt;•\t享勞健保&lt;br&gt;&lt;br&gt;◎休假制度&lt;br&gt;•\t每年 14 天給薪假&lt;br&gt;員工認股,入股分紅,不扣薪病假,部門聚餐,不用補班&lt;br&gt;&lt;br&gt;&lt;br&gt;更多工作資訊請參考：https://www.104.com.tw/job/82ua3', 'employmentType': 'FULL_TIME', 'hiringOrganization': {'@type': 'Organization', 'name': '奇策智能雲端股份有限公司', 'sameAs': 'https://www.104.com.tw/company/1a2x6bliq7', 'logo': 'https://static.104.com.tw/b_profile/cust_picture/5135/130000000165135/logo.png?v=20210306085049'}, 'identifier': {'@type': 'PropertyValue', 'name': '104人力銀行_一零四資訊科技股份有限公司', 'value': '82ua3'}, 'jobLocation': {'@type': 'Place', 'address': {'@type': 'PostalAddress', 'streetAddress': '遠端工作', 'addressLocality': '台北市中正區', 'postalCode': '100', 'addressRegion': '台灣', 'addressCountry': 'TW'}}, 'baseSalary': {'@type': 'MonetaryAmount', 'currency': 'TWD', 'value': {'@type': 'QuantitativeValue', 'unitText': 'DAY', 'value': '論件計酬0元'}}, 'mainEntityOfPage': {'@type': 'WebPage', '@id': '//www.104.com.tw/job/82ua3'}, 'educationRequirements': {'@type': 'EducationalOccupationalCredential', 'credentialCategory': 'high school'}, 'industry': '電腦系統整合服務業', 'workHours': '日班/晚班/假日班，彈性配合', 'skills': ['軟體程式設計', '網路程式設計']}]

In [238]:
data_list1

[{'@context': 'http://schema.org',
  '@type': 'BreadcrumbList',
  'itemListElement': [{'@type': 'ListItem',
    'position': 1,
    'name': '104人力銀行',
    'item': 'https://www.104.com.tw/'},
   {'@type': 'ListItem',
    'position': 2,
    'name': '雲育鏈股份有限公司',
    'item': 'https://www.104.com.tw/company/1a2x6bld2j'},
   {'@type': 'ListItem',
    'position': 3,
    'name': 'Python 開發工程師(台北)',
    'item': 'https://www.104.com.tw/job/87e35'}]},
 {'@context': 'http://schema.org',
  '@type': 'WebPage',
  'name': 'Python 開發工程師(台北)',
  'url': '//www.104.com.tw/job/87e35',
  'relatedLink': 'https://www.104.com.tw/company/1a2x6bld2j',
  'publisher': {'@type': 'Organization', 'name': '雲育鏈股份有限公司'},
  'significantLink': ['https://www.104.com.tw/company/1a2x6bld2j#info06']},
 {'@context': 'http://schema.org',
  '@type': 'JobPosting',
  'title': 'Python 開發工程師(台北)',
  'url': '//www.104.com.tw/job/87e35?jobsource=google_job',
  'datePosted': '2024/01/16',
  'validThrough': '2025-01-17',
  'description':

In [291]:
test = jobs104.get_job_related_mulitline(all_infomation,"【工作內容】","\n- 職務類別：")
print(test)

- 徵求在學學生或應屆畢業生
- 區塊鏈相關 WEB APP 開發與維護。
- ITRI、Apple、Tesla 技術高手親自指導，幫助你學習國外最新 SaaS 開發的觀念與思維。
- 學習量化交易知識與免費使用公司自動交易服務。
- 表現優異者，有機會轉 Paid Intern 或正職！
※ 請注意：此為學習型無薪實習，為期三個月，可配合學校簽訂契約、實習證明和推薦信（表現需具備一定水準者），無法接受者，請勿投遞，謝謝！
※ 需通過遠端線下 coding 測驗（CSS + React 各一題）


In [273]:
print(all_infomation)

【工作內容】
- 徵求在學學生或應屆畢業生
- 區塊鏈相關 WEB APP 開發與維護。
- ITRI、Apple、Tesla 技術高手親自指導，幫助你學習國外最新 SaaS 開發的觀念與思維。
- 學習量化交易知識與免費使用公司自動交易服務。
- 表現優異者，有機會轉 Paid Intern 或正職！
※ 請注意：此為學習型無薪實習，為期三個月，可配合學校簽訂契約、實習證明和推薦信（表現需具備一定水準者），無法接受者，請勿投遞，謝謝！
※ 需通過遠端線下 coding 測驗（CSS + React 各一題）
- 職務類別：Internet程式設計師,軟體工程師
- 工作待遇：論件計酬0元
- 工作性質：全職
- 上班地點：台北市中正區
- 管理責任：不需負擔管理責任
- 出差外派：無需出差外派
- 上班時段：日班/晚班/假日班，彈性配合
- 休假制度：週休二日
- 可上班日：不限
- 需求人數：不限
【相關條件】
- 工作經歷：不拘
- 學歷要求：高中以上
- 公司產業：電腦系統整合服務業
- 科系要求：資訊工程相關,其他數學及電算機科學相關
- 語文條件：英文--聽 /略懂、說 /略懂、讀 /精通、寫 /中等
- 擅長工具：Github,JavaScript,CSS,ReactJS
- 工作技能：軟體程式設計,網路程式設計
【其他條件】
- 其他：對以下領域有興趣者，歡迎應徵：
- Advanced Web Front-end Development
- Blockchain and Smart Contract
- Microservice Technology
- Machine Learning
- Cybersecurity
- 加密貨幣投資
實習期間，你可以學習到：
- 技術分析
- 量化交易自動化
- 程式交易策略開發
- Agile / Scrum 開發實戰
- 前端原子設計框架
- React.js (Next.js)
- HTML / CSS (SCSS)
- TypeScript
- Node.js (ES6)
- Apollo GraphQL Client (react-graphql)
- ...
【公司福利】
- 福利制度：◎薪酬
•	具市場競爭性的薪資
•	員工免費股票
•	員工低價認股權
•	員工私募輪認購 Crypto-Arsen

In [303]:
all_infomation = jobs104.get_all_infomation(data_list2)
job_description = jobs104.get_job_description(all_infomation)
job_category = jobs104.get_job_related(all_infomation,"職務類別")
job_salary = jobs104.get_job_related(all_infomation,"工作待遇")
job_type = jobs104.get_job_related(all_infomation,"工作性質")
job_location = jobs104.get_job_related(all_infomation,"上班地點")
job_responsibility = jobs104.get_job_related(all_infomation,"管理責任")
job_business_trip = jobs104.get_job_related(all_infomation,"出差外派")
job_office_hours = jobs104.get_job_related(all_infomation,"上班時段")
job_vacation = jobs104.get_job_related(all_infomation,"休假制度")
job_available_start = jobs104.get_job_related(all_infomation,"可上班日")
job_vacancy = jobs104.get_job_related(all_infomation,"需求人數")

job_work_experience = jobs104.get_job_related(all_infomation,"工作經歷")
job_educational_requirements = jobs104.get_job_related(all_infomation,"學歷要求")
job_major_requirements = jobs104.get_job_related(all_infomation,"科系要求")
job_language_proficiency = jobs104.get_job_related(all_infomation,"語文條件")
job_tools_proficiency = jobs104.get_job_related(all_infomation,"擅長工具")
job_skills = jobs104.get_job_related(all_infomation,"工作技能")

job_additional_qualifications = jobs104.get_job_related_mulitline(all_infomation,"【其他條件】","【公司福利】")
job_company_benefits = jobs104.get_job_related_mulitline(all_infomation,"【公司福利】","更多工作資訊請參考")
# print(1,job_description)
# print(2,job_category)
# print(3,job_salary)
# print(4,job_type)
# print(5,job_location)
# print(6,job_responsibility)
# print(7,job_business_trip)
# print(8,job_office_hours)
# print(9,job_vacation)
# print(10,job_available_start)
# print(11,job_vacancy)
# print(12,job_work_experience)
# print(13,job_educational_requirements)
# print(14,job_major_requirements)
# print(15,job_language_proficiency)
# print(16,job_tools_proficiency)
# print(17,job_skills)
# print(18,job_additional_qualifications)
print(19,job_company_benefits)

19 - 福利制度：◎薪酬
•	具市場競爭性的薪資
•	員工免費股票
•	員工低價認股權
•	員工私募輪認購 Crypto-Arsenal Token 計畫
◎工作、學習與生活並重
•	遠距工作為主、定期面對面工作計畫與檢討為輔
•	定期部門聚餐
•	線上課程補助
◎保險
•	享勞健保
◎休假制度
•	每年 14 天給薪假
員工認股,入股分紅,不扣薪病假,部門聚餐,不用補班


In [217]:
importlib.reload(async_example)

<module 'async_example' from 'C:\\Users\\Rekam\\Documents\\Jobscan\\async_example.py'>

In [218]:
# # filter_jobs = filter_jobs_104[5:8]
filter_jobs = filter_jobs_104[0:20]
filter_jobs
updated_list = [f"https:{item['href']}" for item in filter_jobs]
updated_list
# # html_content = await jobs104.get_info(filter_job)
# jobs = await async_job.scraper(filter_jobs)

results = await async_example.main(updated_list)

花費 1.42 秒
糖蛙線上娛樂股份有限公司 | JAVA後端工程師 | 2024/01/09
花費 0.48 秒
思銳生醫科技股份有限公司 | 軟體工程師 | 2024/01/11
花費 0.52 秒
星芽社會企業股份有限公司 | 後端工程師（Back-End Developer） | 2024/01/15
花費 0.26 秒
No script tag found.
Error processing link https://www.104.com.tw/job/84qkq?jobsource=n104bank2: the JSON object must be str, bytes or bytearray, not NoneType
花費 0.88 秒
天矽科技股份有限公司 | 網站後端工程師(新北市) | 2024/01/15
花費 0.25 秒
No script tag found.
Error processing link https://www.104.com.tw/job/7iedv?jobsource=n104bank2: the JSON object must be str, bytes or bytearray, not NoneType
花費 0.34 秒
長佳智能股份有限公司 | RD 後端工程師 | 2024/01/15
花費 0.76 秒
資拓宏宇國際股份有限公司 | Java後端工程師-政府領域(PS250) | 2024/01/15
花費 0.68 秒
數位無限軟體股份有限公司 | 後端工程師【台北】 | 2024/01/15
花費 0.38 秒
數位無限軟體股份有限公司 | 後端工程師【遠端】 | 2024/01/15
花費 0.35 秒
長佳智能股份有限公司 | 維運兼後端工程師 | 2024/01/15
花費 0.38 秒
寬橋有限公司 | 產品後端工程師 | 2024/01/15
花費 0.63 秒
升活商務顧問有限公司 | 後端工程師(Backend Engineer)－後端 (System Developer - Backend) #系統發展 | 2024/01/15
花費 0.37 秒
英仕國際有限公司 | 後端工程師 | 2024/01/15
花費 0.6 秒
立炘數位行銷有限公司 | 後端工程師 | 202

In [20]:
results

In [81]:
soup99 = BeautifulSoup(jobs[0], 'html.parser')
text_content = get_title(soup99)


簡單智慧科技股份有限公司 | AI軟體開發工程師 | 2024/01/12 | 【工作內容】
工作內容著重在開發以下深度學習,機器學習跟影像應用的軟體與服務,所需要的能力如下:
1.具備深度學習與機器學習相關知識
2.具備電腦視覺相關知識
3.熟悉C/C+城市開發或有相關開發經驗(與4.擇一即可)
4.熟悉網路服務開發或有相關經驗;主要著重於網站後台與資料庫應用(與3.擇一即可)
5.熟悉Git
6.熟悉Docker(Optional)
7.熟悉Linux開發環境(Optional)
8.對深度學習跟演算法有興趣,能閱讀理解論文內容
9.對於理解問題的本質與解決問題有熱情
以上1到4點為基本要求,5到9點為非必需但對評估工作內容是否適任有加分的作用;另外根據能力的高低與專精會提供不同的職位薪資
- 職務類別：AI工程師,軟體工程師
- 工作待遇：月薪50,000~80,000元
- 工作性質：全職
- 上班地點：台北市中山區
- 管理責任：不需負擔管理責任
- 出差外派：無需出差外派
- 上班時段：日班
- 休假制度：週休二日
- 可上班日：不限
- 需求人數：1人
【相關條件】
- 工作經歷：不拘
- 學歷要求：大學、碩士
- 公司產業：其他電信及通訊相關業
- 科系要求：資訊工程相關,其他數學及電算機科學相關
- 語文條件：不拘
- 擅長工具：不拘
- 工作技能：不拘
【其他條件】
- 其他：專業技能/執照 1.C/C++   2.Python   3.網路服務開發相關程式語言
【公司福利】
- 福利制度：1、周休二日，按照行政機關辦公日曆休假。
2、上班時間：彈性上下班8:30-9:30；18:00-19:00 (午休12:00-13:30)。
3、三節禮品、員工聚餐。
4、依公司營運狀況及個人績效表現，發放年終獎金。
5、其餘未述事宜比照勞基法。
咖啡吧,慶生會,下午茶
更多工作資訊請參考：https://www.104.com.tw/job/85wgm


In [55]:
# 使用 BeautifulSoup 解析 HTML
from bs4 import BeautifulSoup

soup99 = BeautifulSoup(html_content, 'html.parser')
text_content = get_title(soup99)

簡單智慧科技股份有限公司 | AI軟體開發工程師 | 2024/01/12 | 【工作內容】
工作內容著重在開發以下深度學習,機器學習跟影像應用的軟體與服務,所需要的能力如下:
1.具備深度學習與機器學習相關知識
2.具備電腦視覺相關知識
3.熟悉C/C+城市開發或有相關開發經驗(與4.擇一即可)
4.熟悉網路服務開發或有相關經驗;主要著重於網站後台與資料庫應用(與3.擇一即可)
5.熟悉Git
6.熟悉Docker(Optional)
7.熟悉Linux開發環境(Optional)
8.對深度學習跟演算法有興趣,能閱讀理解論文內容
9.對於理解問題的本質與解決問題有熱情
以上1到4點為基本要求,5到9點為非必需但對評估工作內容是否適任有加分的作用;另外根據能力的高低與專精會提供不同的職位薪資
- 職務類別：AI工程師,軟體工程師
- 工作待遇：月薪50,000~80,000元
- 工作性質：全職
- 上班地點：台北市中山區
- 管理責任：不需負擔管理責任
- 出差外派：無需出差外派
- 上班時段：日班
- 休假制度：週休二日
- 可上班日：不限
- 需求人數：1人
【相關條件】
- 工作經歷：不拘
- 學歷要求：大學、碩士
- 公司產業：其他電信及通訊相關業
- 科系要求：資訊工程相關,其他數學及電算機科學相關
- 語文條件：不拘
- 擅長工具：不拘
- 工作技能：不拘
【其他條件】
- 其他：專業技能/執照 1.C/C++   2.Python   3.網路服務開發相關程式語言
【公司福利】
- 福利制度：1、周休二日，按照行政機關辦公日曆休假。
2、上班時間：彈性上下班8:30-9:30；18:00-19:00 (午休12:00-13:30)。
3、三節禮品、員工聚餐。
4、依公司營運狀況及個人績效表現，發放年終獎金。
5、其餘未述事宜比照勞基法。
咖啡吧,慶生會,下午茶
更多工作資訊請參考：https://www.104.com.tw/job/85wgm


In [ ]:
import asyncio
import nest_asyncio
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 在 Jupyter 中處理異步事件循環的嵌套
nest_asyncio.apply()
import asyncio

async def scrape_job_data(item):
    start_time = time.time()

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }

    url = f"https:{item['href']}"
    url = re.sub(r'\?.*$', '', url)

    # 启动 Chrome 浏览器
    option = Options()
    option.page_load_strategy = 'none'
    option.add_argument(f"user-agent={headers['User-Agent']}")
    
    browser = webdriver.Chrome(options=option)
    
    try:
        # 直接使用 await，因為 browser.get 和 browser.wait 本身就是異步的
        await browser.get(url)
        await browser.wait(EC.presence_of_element_located((By.ID, 'app')), timeout=10)
        # 获取加载后的页面内容
        html_content = browser.page_source
    finally:
        # 关闭浏览器
        browser.quit()

    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    # 測試列印
    get_title(soup)
    # 打印输出漂亮格式化的 HTML
    # print(soup.prettify())
    print(f"花費 {round((time.time() - start_time),2)} 秒")
    return soup

# Assuming you have a list of jobs to scrape
filter_jobs = filter_jobs_104[5:8]

# 使用異步方式運行
async def main():
    # 設定並發數量
    max_concurrent = 5
    
    # 使用異步方式運行
    tasks = [scrape_job_data(item) for item in filter_jobs]
    return await asyncio.gather(*tasks)

# result  = await main()  # 直接调用 main() 而不使用 asyncio.gather()

# 創建事件循環
loop = asyncio.get_event_loop()
# 運行主函數
loop.run_until_complete(main())

In [24]:
import async_job
import jobs104
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# result = await async_job.scraper(filter_jobs_104)
# filter_jobs_104
item = filter_jobs_104[20]
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }
# item = "https://www.104.com.tw/job/822rt"
# await jobs104.get_info(filter_jobs_104[0])
from aiohttp import ClientSession, TCPConnector
import pandas as pd
from bs4 import BeautifulSoup

# async def fetch(session, url):
#         async with session.get(url, headers = {'User-Agent':'GoogleBot'}) as response:
#             return await response.text()
# 启动 Chrome 浏览器
option = Options()
option.page_load_strategy = 'none'
option.add_argument(f"user-agent={headers['User-Agent']}")
# option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless') # 瀏覽器不提供頁面觀看，linux下如果系統是純文字介面不加這條會啓動失敗
# option.add_argument('--disable-dev-shm-usage') # 使用共享內存RAM
# option.add_argument('--disable-gpu') # 規避部分chrome gpu bug
# option.add_experimental_option("prefs", prefs)
# option.add_argument('blink-settings=imagesEnabled=false') #不加載圖片提高效率

async def fetch(session,url):
    async with session.get(url, headers = {'User-Agent':'GoogleBot'}) as response:
        return await response.text()



async def main():
    title = item['title']
    print(1, title)
    url = f"https:{item['href']}"
    url = re.sub(r'\?.*$', '', url)

    print(1, url)

    browser = webdriver.Chrome(options=option)
    # 打开网页
    browser.get(url)
    # 显式等待，等待页面加载完成
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'app')))
    # 获取加载后的页面内容
    html_content = browser.page_source
    # 关闭浏览器
    browser.quit()
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(html_content, 'html.parser')



import asyncio
# soup = await asyncio.gather(main())
soup = await main()  # 直接调用 main() 而不使用 asyncio.gather()
# print(3, soup)
# print(1,title)
# print(2,job_link)
# print(3,response)
# print(4,soup)

# data-v-a9f35207 class
# data-v-a9f35207

1 (Jr/Sr.) 後端工程師 Backend Engineer (PHP / Laravel)
1 https://www.104.com.tw/job/6qlwp


# 多執行緒

In [83]:
from concurrent.futures import ThreadPoolExecutor
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Assuming filter_jobs_104 and np are defined elsewhere in your code

def scrape_job_data(item):
    start_time = time.time()

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }

    url = f"https:{item['href']}"
    url = re.sub(r'\?.*$', '', url)

    # 启动 Chrome 浏览器
    option = Options()
    option.page_load_strategy = 'none'
    option.add_argument(f"user-agent={headers['User-Agent']}")
    
    with webdriver.Chrome(options=option) as browser:
        # 打开网页
        browser.get(url)
        # 显式等待，等待页面加载完成
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'app')))
        # 获取加载后的页面内容
        html_content = browser.page_source
    
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    # 打印输出漂亮格式化的 HTML
    # print(soup.prettify())
    print(f"花費 {np.round((time.time() - start_time),2)} 秒")

# Assuming you have a list of jobs to scrape
filter_jobs= filter_jobs_104[0:5]

# Set the maximum number of concurrent threads
max_threads = 5

# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_threads) as executor:
    # Submit each job for scraping concurrently
    futures = [executor.submit(scrape_job_data, item) for item in filter_jobs]

    # Wait for all futures to complete
    for future in futures:
        future.result()


花費 6.36 秒
花費 6.44 秒
花費 7.12 秒
花費 7.13 秒
花費 7.75 秒


In [84]:
import asyncio
import nest_asyncio
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 在 Jupyter 中處理異步事件循環的嵌套
nest_asyncio.apply()

async def scrape_job_data(item):
    start_time = time.time()

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }

    url = f"https:{item['href']}"
    url = re.sub(r'\?.*$', '', url)

    # 启动 Chrome 浏览器
    option = Options()
    option.page_load_strategy = 'none'
    option.add_argument(f"user-agent={headers['User-Agent']}")
    
    browser = webdriver.Chrome(options=option)
    
    try:
        # 打开网页
        browser.get(url)
        # 显式等待，等待页面加载完成
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'app')))
        # 获取加载后的页面内容
        html_content = browser.page_source
    finally:
        # 关闭浏览器
        browser.quit()

    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    # 測試列印
    get_title(soup)
    # 打印输出漂亮格式化的 HTML
    # print(soup.prettify())
    print(f"花費 {round((time.time() - start_time),2)} 秒")

# Assuming you have a list of jobs to scrape
filter_jobs = filter_jobs_104[0:5]

# 使用異步方式運行
async def main():
    # 設定並發數量
    max_concurrent = 5
    
    # 使用異步方式運行
    tasks = [scrape_job_data(item) for item in filter_jobs]
    await asyncio.gather(*tasks)

# 創建事件循環
loop = asyncio.get_event_loop()
# 運行主函數
loop.run_until_complete(main())


花費 4.17 秒
花費 4.42 秒
花費 4.07 秒
花費 4.09 秒
花費 4.12 秒


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time




from bs4 import BeautifulSoup
start_time = time.time()

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }


item = filter_jobs_104[120]
url = f"https:{item['href']}"
url = re.sub(r'\?.*$', '', url)

# 启动 Chrome 浏览器
option = Options()
option.page_load_strategy = 'none'
option.add_argument(f"user-agent={headers['User-Agent']}")
# option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_argument('--headless') # 瀏覽器不提供頁面觀看，linux下如果系統是純文字介面不加這條會啓動失敗
# option.add_argument('--disable-dev-shm-usage') # 使用共享內存RAM
# option.add_argument('--disable-gpu') # 規避部分chrome gpu bug
# option.add_experimental_option("prefs", prefs)
# option.add_argument('blink-settings=imagesEnabled=false') #不加載圖片提高效率

browser = webdriver.Chrome(options=option)
# 打开网页
browser.get(url)
# 显式等待，等待页面加载完成
WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'app')))
# 获取加载后的页面内容
html_content = browser.page_source
# 关闭浏览器
browser.quit()
# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(html_content, 'html.parser')
# soup = BeautifulSoup(response, 'lxml')
# 打印输出漂亮格式化的 HTML
# print(soup.prettify())
print(f"花費 {np.round((time.time() - start_time),2)} 秒")


In [76]:
import json
import html


def get_title(soup):
    if soup is None:
        print("No data available.")
        return None

    # 找到包含JavaScript代码的script标签
    script_tag = soup.find('script', type='application/ld+json')

    if script_tag is None:
        print("No script tag found.")
        return None

    json_string = script_tag.text

    # 将JSON字符串转换为Python的list
    data_list = json.loads(json_string)

    # 打印转换后的list
    company = data_list[0]['itemListElement'][1]['name']
    print(company, end=" | ")

    position = data_list[0]['itemListElement'][2]['name']
    print(position, end=" | ")

    update = data_list[2]['datePosted']
    print(update, end=" | ")

    description = data_list[2]['description']
    description = html.unescape(description)

    # 使用 BeautifulSoup 解析 HTML
    soup_descript = BeautifulSoup(description, 'html.parser')

    # 将 <br> 转换成换行符号
    for br_tag in soup_descript.find_all('br'):
        br_tag.replace_with('\n')

    # 提取纯文本内容
    text_content = soup_descript.get_text(separator='\n', strip=True)
    print(text_content)
    return text_content
    
    
    # 打印排版後的文本
    # print(text_content)


In [74]:
# soup
# def update_date(self, soup) -> str:
# update_date = soup.find("div", class_="job-header__title")
soup.prettify()
# soup = BeautifulSoup(html_content, 'html.parser')

# 使用 prettify() 方法格式化输出
formatted_html = soup.prettify()

# 打印输出
print(formatted_html)
# update_date = soup.find("ul", class_="global_nav")
# print(update_date)
        # return update_date.find('span').text.strip().replace('更新','') if update_date else '無'

<!DOCTYPE html>
<html lang="zh-Hant-TW">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1.0" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <link href="android-app://com.m104/m104/job?j=4b514a753f7350274b49422459353d6643f4149723b3d456e434941242424246974d41497513j98&amp;jobNo=822rt" rel="alternate"/>
  <link href="ios-app://437817158/m104/job?j=4b514a753f7350274b49422459353d6643f4149723b3d456e434941242424246974d41497513j98&amp;jobNo=822rt" rel="alternate"/>
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://static.104.com.tw" rel="dns-prefetch"/>
  <link href="//cdn.104.com.tw" rel="dns-prefetch"/>
  <!-- Google Tag Manager -->
  <script async="" src="https://connect.facebook.net/en_US/fbevents.js">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-820745912&amp;l=dataLayer&amp;cx=c" type="text/javascri

# 參考Code

In [2]:
class eJob_search104():
    
    
    def update_date(self, soup) -> str:
        update_date = soup.find("div", class_="job-header__title")
        return update_date.find('span').text.strip().replace('更新','') if update_date else '無'
    
    def company(self, soup) -> str:
        name = soup.find("div", class_="mt-3")
        return name.select_one('div > a').text.strip() if name else '無'
    
    def jd_info(self, soup) -> dict:
        result = {}
        JD = soup.find('div', class_='job-description-table row')
        if JD:
            jd_items = JD.find_all('div', recursive=False)
            if jd_items:
                    try:
                        job_content = jd_items[0].find('p').text if jd_items[0].find('p').text else '無'
                    except:
                        job_content = '無'

                    try:
                        job_category = ', '.join(i.text for i in jd_items[1].find_all('u')) if jd_items[1].find_all('u') else '無'
                    except:
                        job_category = '無'
                    
                    try:
                        salary = jd_items[2].find_all('div', recursive=False)[-1].text.strip() if jd_items[2].find_all('div', recursive=False)[-1].text.strip() else '無'
                    except:
                        salary = '無'
                    
                    try:
                        job_type = jd_items[3].find_all('div')[-1].text.strip() if jd_items[3].find_all('div')[-1].text.strip() else '無'
                    except:
                        job_type = '無'

                    try:
                        workingplace = jd_items[4].find_all('div')[-1].text.strip() if jd_items[4].find_all('div')[-1].text.strip() != '代企業徵才' else None
                    except:
                        workingplace = '無'

                    try:
                        management_responsibility = jd_items[6].find_all('div')[-1].text.strip() if jd_items[6].find_all('div')[-1].text.strip() else '無'
                    except:
                        management_responsibility = '無'
                    
                    try:
                        business_trip = jd_items[7].find_all('div')[-1].text.strip() if jd_items[7].find_all('div')[-1].text.strip() else '無'
                    except:
                        business_trip = '無'

                    try:
                        working_duration = jd_items[8].find_all('div')[-1].text.strip() if jd_items[8].find_all('div')[-1].text.strip() else '無'
                    except:
                        working_duration = '無'
                    
                    try:
                        Holiday_System = jd_items[9].find_all('div')[-1].text.strip() if jd_items[9].find_all('div')[-1].text.strip() else '無'
                    except:
                        Holiday_System = '無'
                    
                    try:
                        Working_date = jd_items[10].find_all('div')[-1].text.strip() if jd_items[10].find_all('div')[-1].text.strip() else '無'
                    except:
                        Working_date = '無'
                    
                    try:
                        ppl_required = jd_items[11].find_all('div')[-1].text.strip() if jd_items[11].find_all('div')[-1].text.strip() else '無'
                    except:
                        ppl_required = '無'


                    result = {
                                '工作內容': job_content,
                                '職務類別': job_category,
                                '工作待遇': salary,
                                '工作性質': job_type,
                                '上班地點': workingplace,
                                '管理責任': management_responsibility,
                                '出差外派': business_trip,
                                '上班時段': working_duration,
                                '休假制度': Holiday_System,
                                '可上班日': Working_date,
                                '需求人數': ppl_required
                            }
        return result
    
    def jr_info(self, soup) -> dict:
        result = {}
        JR = soup.find('div', class_= 'job-requirement-table row')
        JRO = soup.find('div', class_= 'job-requirement col opened')
        if JR:
            jr_items = JR.find_all('div', recursive=False)
            if jr_items:
                try:
                    work_exp = jr_items[0].find_all('div')[-1].text.strip() if jr_items[0].find_all('div')[-1].text.strip() else '無'
                except:
                    work_exp = '無'
                
                try:
                    academic_require = jr_items[1].find_all('div')[-1].text.strip() if jr_items[1].find_all('div')[-1].text.strip() else '無'
                except:
                    academic_require = '無'
                
                try:
                    department_require = jr_items[2].find_all('div')[-1].text.strip() if jr_items[2].find_all('div')[-1].text.strip() else '無'
                except:
                    department_require = '無'

                try:
                    language = jr_items[3].find('p').text.strip() if jr_items[3].find('p').text.strip() else '無'
                except:
                    language = '無'

                try:
                    tool = ', '.join(i.text for i in jr_items[4].find_all('u')) if jr_items[4].find_all('u') else '無'
                except:
                    tool = '無'
                
                try:
                    working_ability = jr_items[5].find_all('div')[-1].text.strip() if jr_items[5].find_all('div')[-1].text.strip() else '無'
                except:
                    working_ability = '無'
   
        try:
            others = JRO.find_all('div')[-1].text.strip() if JRO.find_all('div')[-1].text.strip() else '無'
        except:
            others = '無'

        if JR and JRO:
            result = {
                '工作經歷' : work_exp,
                '學歷要求' : academic_require,
                '科系要求' : department_require,
                '語文條件' : language,
                '擅長工具' : tool,
                '工作技能' : working_ability,
                '其他要求' : others
            }

        # result = {
        #     '工作經歷' : work_exp,
        #     '學歷要求' : academic_require,
        #     '科系要求' : department_require,
        #     '語文條件' : language,
        #     '擅長工具' : tool,
        #     '工作技能' : working_ability,
        #     '其他要求' : others
        # }
        return result
    
    async def fetch(self, session, url):
        async with session.get(url, headers = {'User-Agent':'GoogleBot'}) as response:
            return await response.text()

    async def get_job_info(self, item):
        try:
            title = item['title']
            Job_link = f"https:{item['href']}"
            connector = TCPConnector(limit=10)
            async with ClientSession(connector=connector) as session:
                html = await self.fetch(session, Job_link)
                soup = BeautifulSoup(html, 'lxml')

            Data = {
                '更新日期': [self.update_date(soup)],
                '職缺名稱': [title],
                '公司名稱': [self.company(soup)],
                '連結': [Job_link]
            }
            Data.update(self.jd_info(soup))
            Data.update(self.jr_info(soup))
            df = pd.DataFrame(Data, columns=['更新日期', '職缺名稱', '公司名稱', '工作內容', '職務類別', '工作待遇',
                                            '工作性質', '上班地點', '管理責任', '出差外派', '上班時段', '休假制度',
                                            '可上班日', '需求人數', '工作經歷', '學歷要求', '科系要求', '語文條件',
                                            '擅長工具', '工作技能', '其他要求', '連結'])
            return df
        except Exception as e:
            print("發生錯誤", e)
            return None

    async def scrape(self, Job_list):
        tasks = []
        semaphore = asyncio.Semaphore(10)  # Limit concurrent requests to 10

        for item in Job_list:
            async with semaphore:
                task = asyncio.ensure_future(self.get_job_info(item))
                tasks.append(task)

        return await asyncio.gather(*tasks)
    
    def main(self, Job_list: list, DF):
        print(len(Job_list))
        batch_size = 30
        num_batches = (len(Job_list) + batch_size - 1) // batch_size
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(Job_list))
            Job_list_batch = Job_list[start_idx:end_idx]
            loop = asyncio.get_event_loop()
            results = loop.run_until_complete(self.scrape(Job_list_batch))
            # loop.close()

            for df in results:
                if df is not None:
                    DF = pd.concat([DF, df], ignore_index=True)

        DF.to_csv(f"./output/JBLIST_{self.current_date}.csv", sep=',', index=False)
        return DF

# 測試範例

In [8]:
# 如果你在一個 Python script 中執行這樣的程式碼，你可能需要使用 asyncio.run() 函數。
# 在異步程式碼中，如果你使用 asyncio.gather 函數收集異步任務的結果，而這些任務沒有顯式返回值，
# gather 函數將返回一個包含每個異步任務結果的列表，而這些結果通常是 None。

import asyncio

async def async_example(i):
    print(f"Start asynchronous task {i}")
    await asyncio.sleep(1)
    print(f"End asynchronous task {i}")
    return f"Result of task {i}"

# 直接在 Jupyter cell 中執行
tasks = [async_example(i) for i in range(5)]
results = await asyncio.gather(*tasks)
print(results)

Start asynchronous task 0
Start asynchronous task 1
Start asynchronous task 2
Start asynchronous task 3
Start asynchronous task 4
End asynchronous task 0
End asynchronous task 2
End asynchronous task 4
End asynchronous task 1
End asynchronous task 3
['Result of task 0', 'Result of task 1', 'Result of task 2', 'Result of task 3', 'Result of task 4']


In [6]:
# 執行 async_example.py 範例
from async_example import main
result = await main()


338


Batch 12 - (331-338): 100%|██████████| 8/8 [00:01<00:00,  7.93task/s]


In [1]:
import pandas as pd
import re, time, requests, random
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# import grequests # 看起要在.py才能用
from aiohttp import ClientSession, TCPConnector
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
class eJob_search104():
    current_date = datetime.now().date()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
    }
    search_url = 'https://www.104.com.tw/jobs/search/?'
    def __init__(self, filter_params, key_word, page = 15):
        self.filter_params = filter_params
        self.key_word = key_word
        self.page = page
           
    def search_job(self):
        url = requests.get(self.search_url, self.filter_params, headers=self.headers).url
        print(url)
        option = Options()
        option.add_argument(f"user-agent={self.headers['User-Agent']}")
        option.add_experimental_option('excludeSwitches', ['enable-automation']) # 開發者模式。可以避開某些防爬機制，有開有保佑
        # option.add_argument('--headless') # 無頭模式，開發完成之後再使用，可以完全背景執行，有機會變快
        # option.add_argument("--disable-gpu") # 禁用GPU加速，有些情況下需要設置這個參數
        driver = webdriver.Chrome(options=option)
        driver.get(url)

        element = driver.find_element(By.XPATH,'//*[@id="js-job-header"]/div[1]/label[1]/select/option[1]')
        total_page = int(re.sub(r'\D', '', element.text.split('/')[-1]))
        print(f'Total_page = {total_page}')
        # 滾頁面
        scroll_times = self.page
        for _ in range(scroll_times):
            driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
            time.sleep(2)

        # 自動加載結束後要自行點選載入(15以後)
        # 使用CSS選擇器定位最後一個按鈕並點擊
        if total_page >= 15:
            k = 1
            while True:
                try:
                    button_element = WebDriverWait(driver, 4).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, '#js-job-content > div:last-child > button'))
                    )
                    print(f'手動載入第{15 + k}頁')
                    button_element.click()
                    k += 1
                    if k == 86 or k == total_page - 14 :
                        break
                except Exception as e:
                    print("發生未知錯誤：", e)
                    break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        raw_Job_list = soup.find_all("a",class_="js-job-link")
        print(f'共{len(raw_Job_list)}筆資料')
        driver.quit()
        return raw_Job_list
    
    def filter_job(self, raw_Job_list:list):
        filter_job_list = [i for i in raw_Job_list if re.search(self.key_word, i['title'].lower())]
        print(f'過濾完有{len(filter_job_list)}筆')
        return filter_job_list
    
    def update_date(self, soup) -> str:
        update_date = soup.find("div", class_="job-header__title")
        return update_date.find('span').text.strip().replace('更新','') if update_date else '無'
    
    def company(self, soup) -> str:
        name = soup.find("div", class_="mt-3")
        return name.select_one('div > a').text.strip() if name else '無'
    
    def jd_info(self, soup) -> dict:
        result = {}
        JD = soup.find('div', class_='job-description-table row')
        if JD:
            jd_items = JD.find_all('div', recursive=False)
            if jd_items:
                    try:
                        job_content = jd_items[0].find('p').text if jd_items[0].find('p').text else '無'
                    except:
                        job_content = '無'

                    try:
                        job_category = ', '.join(i.text for i in jd_items[1].find_all('u')) if jd_items[1].find_all('u') else '無'
                    except:
                        job_category = '無'
                    
                    try:
                        salary = jd_items[2].find_all('div', recursive=False)[-1].text.strip() if jd_items[2].find_all('div', recursive=False)[-1].text.strip() else '無'
                    except:
                        salary = '無'
                    
                    try:
                        job_type = jd_items[3].find_all('div')[-1].text.strip() if jd_items[3].find_all('div')[-1].text.strip() else '無'
                    except:
                        job_type = '無'

                    try:
                        workingplace = jd_items[4].find_all('div')[-1].text.strip() if jd_items[4].find_all('div')[-1].text.strip() != '代企業徵才' else None
                    except:
                        workingplace = '無'

                    try:
                        management_responsibility = jd_items[6].find_all('div')[-1].text.strip() if jd_items[6].find_all('div')[-1].text.strip() else '無'
                    except:
                        management_responsibility = '無'
                    
                    try:
                        business_trip = jd_items[7].find_all('div')[-1].text.strip() if jd_items[7].find_all('div')[-1].text.strip() else '無'
                    except:
                        business_trip = '無'

                    try:
                        working_duration = jd_items[8].find_all('div')[-1].text.strip() if jd_items[8].find_all('div')[-1].text.strip() else '無'
                    except:
                        working_duration = '無'
                    
                    try:
                        Holiday_System = jd_items[9].find_all('div')[-1].text.strip() if jd_items[9].find_all('div')[-1].text.strip() else '無'
                    except:
                        Holiday_System = '無'
                    
                    try:
                        Working_date = jd_items[10].find_all('div')[-1].text.strip() if jd_items[10].find_all('div')[-1].text.strip() else '無'
                    except:
                        Working_date = '無'
                    
                    try:
                        ppl_required = jd_items[11].find_all('div')[-1].text.strip() if jd_items[11].find_all('div')[-1].text.strip() else '無'
                    except:
                        ppl_required = '無'


                    result = {
                                '工作內容': job_content,
                                '職務類別': job_category,
                                '工作待遇': salary,
                                '工作性質': job_type,
                                '上班地點': workingplace,
                                '管理責任': management_responsibility,
                                '出差外派': business_trip,
                                '上班時段': working_duration,
                                '休假制度': Holiday_System,
                                '可上班日': Working_date,
                                '需求人數': ppl_required
                            }
        return result
    
    def jr_info(self, soup) -> dict:
        result = {}
        JR = soup.find('div', class_= 'job-requirement-table row')
        JRO = soup.find('div', class_= 'job-requirement col opened')
        if JR:
            jr_items = JR.find_all('div', recursive=False)
            if jr_items:
                try:
                    work_exp = jr_items[0].find_all('div')[-1].text.strip() if jr_items[0].find_all('div')[-1].text.strip() else '無'
                except:
                    work_exp = '無'
                
                try:
                    academic_require = jr_items[1].find_all('div')[-1].text.strip() if jr_items[1].find_all('div')[-1].text.strip() else '無'
                except:
                    academic_require = '無'
                
                try:
                    department_require = jr_items[2].find_all('div')[-1].text.strip() if jr_items[2].find_all('div')[-1].text.strip() else '無'
                except:
                    department_require = '無'

                try:
                    language = jr_items[3].find('p').text.strip() if jr_items[3].find('p').text.strip() else '無'
                except:
                    language = '無'

                try:
                    tool = ', '.join(i.text for i in jr_items[4].find_all('u')) if jr_items[4].find_all('u') else '無'
                except:
                    tool = '無'
                
                try:
                    working_ability = jr_items[5].find_all('div')[-1].text.strip() if jr_items[5].find_all('div')[-1].text.strip() else '無'
                except:
                    working_ability = '無'
   
        try:
            others = JRO.find_all('div')[-1].text.strip() if JRO.find_all('div')[-1].text.strip() else '無'
        except:
            others = '無'

        if JR and JRO:
            result = {
                '工作經歷' : work_exp,
                '學歷要求' : academic_require,
                '科系要求' : department_require,
                '語文條件' : language,
                '擅長工具' : tool,
                '工作技能' : working_ability,
                '其他要求' : others
            }

        # result = {
        #     '工作經歷' : work_exp,
        #     '學歷要求' : academic_require,
        #     '科系要求' : department_require,
        #     '語文條件' : language,
        #     '擅長工具' : tool,
        #     '工作技能' : working_ability,
        #     '其他要求' : others
        # }
        return result
    
    async def fetch(self, session, url):
        async with session.get(url, headers = {'User-Agent':'GoogleBot'}) as response:
            return await response.text()

    async def get_job_info(self, item):
        try:
            title = item['title']
            Job_link = f"https:{item['href']}"
            connector = TCPConnector(limit=10)
            async with ClientSession(connector=connector) as session:
                html = await self.fetch(session, Job_link)
                soup = BeautifulSoup(html, 'lxml')

            Data = {
                '更新日期': [self.update_date(soup)],
                '職缺名稱': [title],
                '公司名稱': [self.company(soup)],
                '連結': [Job_link]
            }
            Data.update(self.jd_info(soup))
            Data.update(self.jr_info(soup))
            df = pd.DataFrame(Data, columns=['更新日期', '職缺名稱', '公司名稱', '工作內容', '職務類別', '工作待遇',
                                            '工作性質', '上班地點', '管理責任', '出差外派', '上班時段', '休假制度',
                                            '可上班日', '需求人數', '工作經歷', '學歷要求', '科系要求', '語文條件',
                                            '擅長工具', '工作技能', '其他要求', '連結'])
            return df
        except Exception as e:
            print("發生錯誤", e)
            return None

    async def scrape(self, Job_list):
        tasks = []
        semaphore = asyncio.Semaphore(10)  # Limit concurrent requests to 10

        for item in Job_list:
            async with semaphore:
                task = asyncio.ensure_future(self.get_job_info(item))
                tasks.append(task)

        return await asyncio.gather(*tasks)
    
    def main(self, Job_list: list, DF):
        print(len(Job_list))
        batch_size = 30
        num_batches = (len(Job_list) + batch_size - 1) // batch_size
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(Job_list))
            Job_list_batch = Job_list[start_idx:end_idx]
            loop = asyncio.get_event_loop()
            results = loop.run_until_complete(self.scrape(Job_list_batch))
            # loop.close()

            for df in results:
                if df is not None:
                    DF = pd.concat([DF, df], ignore_index=True)

        DF.to_csv(f"./output/JBLIST_{self.current_date}.csv", sep=',', index=False)
        return DF

In [3]:
start_time = time.time()

# 過濾關鍵字以外的職缺
keywords_pattern = r'工程|資料|python|data|數據'

# 搜尋關鍵字
filter_params = {
    'ro' : 1, # 1 全職
    'keyword' : '資料工程',
    'area' : '6001002000,6001001000,6001006000,C6001008000',  # 6001001000 台北市 6001002000 新北 6001006000 新竹縣市 6001008000 台中市
    'isnew' : 3, # 0:本日 3:3天內 7:1週內 14 30
    'jobexp' : '1,3', # 工作經驗1年以下 + 1-3年
    'mode' : 'l', # 列表模式(比較多筆資料)
    'order' : 16 # 照日期排序
}

# 建立物件
DF = pd.DataFrame()
EJS = eJob_search104(filter_params, keywords_pattern)
retry_count = 0
while True:
    try:
        raw_Job_list = EJS.search_job()
        break
    except:
        retry_count += 1
        if retry_count == 3:
            break
        print(f'執行錯誤, retry {retry_count}')
Job_list = EJS.filter_job(raw_Job_list)
result_df = EJS.main(Job_list, DF)
print(f"花費 {time.time() - start_time} 秒")

https://www.104.com.tw/jobs/search/?ro=1&keyword=%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B&area=6001002000%2C6001001000%2C6001006000%2CC6001008000&isnew=3&jobexp=1%2C3&mode=l&order=16
Total_page = 65
手動載入第16頁
手動載入第17頁
手動載入第18頁
手動載入第19頁
手動載入第20頁
手動載入第21頁
手動載入第22頁
手動載入第23頁
手動載入第24頁
手動載入第25頁
手動載入第26頁
手動載入第27頁
手動載入第28頁
手動載入第29頁
手動載入第30頁
手動載入第31頁
手動載入第32頁
手動載入第33頁
手動載入第34頁
手動載入第35頁
手動載入第36頁
手動載入第37頁
手動載入第38頁
手動載入第39頁
手動載入第40頁
手動載入第41頁
手動載入第42頁
手動載入第43頁
手動載入第44頁
手動載入第45頁
手動載入第46頁
手動載入第47頁
手動載入第48頁
手動載入第49頁
手動載入第50頁
手動載入第51頁
手動載入第52頁
手動載入第53頁
手動載入第54頁
手動載入第55頁
手動載入第56頁
手動載入第57頁
手動載入第58頁
手動載入第59頁
手動載入第60頁
手動載入第61頁
手動載入第62頁
手動載入第63頁
手動載入第64頁
手動載入第65頁
共1930筆資料
過濾完有1271筆
1271
花費 143.77451586723328 秒
